# Menggunakan Model ETS untuk TAHUN 2025-2026

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing

pd.set_option('display.float_format', lambda x: f'{x:.2f}')

# =========================================================
# 1. LOAD DATA
# =========================================================
df = pd.read_csv("data/DataJepang.csv")

# Drop columns yang tidak dipakai
df = df.drop(columns=["DATASET", "SERIES_CODE", "OBS_MEASURE"])


# =========================================================
# 2. Pastikan kolom tahun 2024–2030 ada
# =========================================================
forecast_years = list(range(2024, 2031))

for y in forecast_years:
    col = str(y)
    if col not in df.columns:
        df[col] = np.nan

# =========================================================
# 3. Ambil kolom historis (2000–2023)
# =========================================================
year_hist = list(range(2000, 2024))  # 2023 final actual
year_cols_hist = [str(y) for y in year_hist]


# =========================================================
# 4. ETS FORECAST 2024–2030
# =========================================================
all_fc = np.zeros((len(df), len(forecast_years)))

for i, (_, row) in enumerate(df.iterrows()):
    series = row[year_cols_hist].astype(float).dropna().values

    if len(series) < 5:
        continue

    model = ExponentialSmoothing(series, trend="add", seasonal=None,
                                 initialization_method="estimated")
    fit = model.fit(optimized=True)

    fc = fit.forecast(len(forecast_years))
    all_fc[i, :] = fc

# Masukkan hasil forecast
for idx, y in enumerate(forecast_years):
    df[str(y)] = all_fc[:, idx]


# =========================================================
# 5. Wide → Long
# =========================================================
all_year_cols = [str(y) for y in (year_hist + forecast_years)]
id_vars = [c for c in df.columns if c not in all_year_cols]

long_df = df.melt(
    id_vars=id_vars,
    value_vars=all_year_cols,
    var_name="Year",
    value_name="Value"
)

long_df["Year"] = long_df["Year"].astype(int)
long_df["Value"] = pd.to_numeric(long_df["Value"], errors="coerce")


# =========================================================
# 6. Tambah kolom Type: Actual / Forecast
# =========================================================
long_df["Type"] = np.where(long_df["Year"] <= 2023, "Actual", "Forecast")


# =========================================================
# 7. Simpan FINAL
# =========================================================
output_path = "data/DataJepangForecast.csv"
long_df.to_csv(output_path, index=False, float_format="%.2f")


long_df.head()


DONE! File Jepang final disimpan ke: data/DataJepangForecast.csv


,COUNTRY,INDICATOR,FREQUENCY,SCALE,Year,Value,Type
0,Japan,"Gross domestic product (GDP), Constant prices,...",Annual,Units,2000,2.57,Actual
1,Japan,"Net lending (+) / net borrowing (-), General g...",Annual,Units,2000,-7.28,Actual
2,Japan,"All Items, Consumer price index (CPI), End-of-...",Annual,Units,2000,-0.77,Actual
3,Japan,"All Items, Consumer price index (CPI), Period ...",Annual,Units,2000,-0.69,Actual
4,Japan,"Gross domestic product (GDP), Constant prices,...",Annual,Units,2000,2.76,Actual
